In [1]:
!pip install pyvis python-louvain


[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import numpy as np
import torch
from community import community_louvain
from networkx.algorithms.community import girvan_newman
from itertools import islice
import random
import seaborn as sns
import re
from collections import Counter
import ast

In [7]:




# data_path="tripAdvisor/task2_graph.gml"
# parquet_path= "tripAdvisor/task2_df.parquet"

G = nx.read_gml('/kaggle/input/sna-project-files/task2_graph.gml')
df = pd.read_parquet('/kaggle/input/sna-project-files/task2_df.parquet')
print(f"Loaded graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Loaded DataFrame: {len(df)} rows")

print(G.number_of_nodes())

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sna-project-files/task2_graph.gml'

In [109]:
# df = pd.read_parquet('./task2_df.parquet')
print(df.columns)

Index(['restaurant_link', 'restaurant_name', 'original_location', 'country',
       'region', 'province', 'city', 'address', 'latitude', 'longitude',
       'claimed', 'awards', 'popularity_detailed', 'popularity_generic',
       'top_tags', 'price_level', 'price_range', 'meals', 'cuisines',
       'special_diets', 'features', 'vegetarian_friendly', 'vegan_options',
       'gluten_free', 'original_open_hours', 'open_days_per_week',
       'open_hours_per_week', 'working_shifts_per_week', 'avg_rating',
       'total_reviews_count', 'default_language',
       'reviews_count_in_default_language', 'excellent', 'very_good',
       'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere',
       'keywords'],
      dtype='object')


In [110]:
print(df['popularity_generic'].isna().sum())
print(df['popularity_generic'])
# print(df['total_reviews_count'])

0
91460                      #1 of 3 places to eat in Suzette
701674         #277 of 458 places to eat in Reggio Calabria
389450            #6650 of 10645 places to eat in Barcelona
566304                                                 None
346212    #11 of 23 places to eat in Santa Lucia de Tira...
                                ...                        
604106                   #5 of 6 places to eat in Glynneath
489262                    #17 of 120 places to eat in Neath
772878                  #69 of 148 places to eat in Albenga
40406                 #6671 of 18480 places to eat in Paris
719098                 #3307 of 12913 places to eat in Rome
Name: popularity_generic, Length: 5000, dtype: object


In [111]:
# Step 1: Load required files
centrality_df = pd.read_csv('./task4_centrality.csv')
df = pd.read_parquet('./task2_df.parquet')
print(f"Loaded centrality data: {len(centrality_df)} rows")
print(f"Loaded restaurant data: {len(df)} rows")
print("Columns in restaurant data:", df.columns.tolist())

FileNotFoundError: [Errno 2] No such file or directory: './task4_centrality.csv'

In [ ]:
# Step 2: Clean restaurant_name for consistent merging
def clean_name(name):
    if pd.isna(name):
        return name
    # Strip spaces, lowercase, remove special characters, normalize spaces
    name = name.strip().lower()
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

centrality_df['restaurant_name'] = centrality_df['restaurant_name'].apply(clean_name)
df['restaurant_name'] = df['restaurant_name'].apply(clean_name)

# Verify common restaurant_names
common_names = len(set(centrality_df['restaurant_name']) & set(df['restaurant_name']))
print(f"Common restaurant_names: {common_names}")
print("Sample centrality restaurant_names:", centrality_df['restaurant_name'].head().tolist())
print("Sample restaurant dataset names:", df['restaurant_name'].head().tolist())

# Debug unmatched names
unmatched = set(centrality_df['restaurant_name']) - set(df['restaurant_name'])
if unmatched:
    print("Sample unmatched centrality names:", list(unmatched)[:5])

# Step 3: Parse popularity_generic into numeric popularity_score (1/rank)
def parse_popularity_generic(text):
    if pd.isna(text) or text is None or text.lower() == 'none':
        return np.nan
    # Extract ranking number (e.g., "#1" -> 1)
    match = re.search(r'#(\d+)', str(text))
    if match:
        rank = int(match.group(1))
        return 1.0 / rank if rank > 0 else np.nan  # Inverse rank for higher popularity = higher score
    return np.nan

df['popularity_score'] = df['popularity_generic'].apply(parse_popularity_generic)
print("\nSample popularity_generic and parsed popularity_score:")
print(df[['popularity_generic', 'popularity_score']].head(10))
print("popularity_score NaN count:", df['popularity_score'].isna().sum())

# Step 4: Identify available metrics
available_metrics = []
for col in ['popularity_score', 'total_reviews_count', 'avg_rating']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        if df[col].isna().sum() < len(df):
            available_metrics.append(col)
    else:
        print(f"Warning: Column {col} not found in task2_df.parquet")
if not available_metrics:
    raise ValueError("No valid metrics (popularity_score, total_reviews_count, avg_rating) found")

print(f"Available metrics: {available_metrics}")

# Step 5: Merge centrality scores with metrics
analysis_df = centrality_df[['restaurant_name', 'degree_centrality', 'closeness_centrality', 
                            'betweenness_centrality', 'eigenvector_centrality']].merge(
    df[['restaurant_name'] + available_metrics],
    on='restaurant_name',
    how='left'
)
print(f"Merged data: {len(analysis_df)} rows")

# Step 6: Handle missing or invalid data
numeric_columns = ['degree_centrality', 'closeness_centrality', 'betweenness_centrality', 
                   'eigenvector_centrality'] + available_metrics
for col in numeric_columns:
    analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

# Check NaN values
print("\nNaN counts in numeric columns:")
print(analysis_df[numeric_columns].isna().sum())

# Impute NaN for metrics
for col in available_metrics:
    if col in ['total_reviews_count']:
        analysis_df[col] = analysis_df[col].fillna(0)
    else:  # popularity_score, avg_rating
        analysis_df[col] = analysis_df[col].fillna(analysis_df[col].mean())

# Drop rows where all centrality scores are NaN
analysis_df = analysis_df.dropna(subset=['degree_centrality', 'eigenvector_centrality'], how='all')
print(f"Data after cleaning: {len(analysis_df)} rows")

# Step 7: Compute Pearson correlations
correlation_matrix = analysis_df[numeric_columns].corr(method='pearson')
print("\nPearson Correlation Matrix:")
print(correlation_matrix)

# Extract relevant correlations
correlations = correlation_matrix.loc[
    ['degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'eigenvector_centrality'],
    available_metrics
]
print("\nCentrality vs. Metrics Correlations:")
print(correlations)

# Step 8: Interpret results
print("\nInterpretations and Implications:")
print("Note: popularity_generic was parsed into popularity_score (1/rank, higher score = higher popularity).")
for centrality in ['degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'eigenvector_centrality']:
    print(f"\n{centrality.replace('_', ' ').title()}:")
    for metric in available_metrics:
        corr = correlations.loc[centrality, metric]
        strength = "strong" if abs(corr) > 0.5 else "moderate" if abs(corr) > 0.3 else "weak"
        direction = "positive" if corr > 0 else "negative"
        print(f"- Correlation with {metric}: {corr:.4f} ({strength}, {direction})")
        print(f"  Implication: {centrality.replace('_', ' ').title()} {'is' if abs(corr) > 0.3 else 'is not'} strongly linked to {metric}.")

# Step 9: Visualize relationships (Scatter Plots)
n_metrics = len(available_metrics)
plt.figure(figsize=(15, 4 * n_metrics))
plot_idx = 1
for centrality in ['degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'eigenvector_centrality']:
    for metric in available_metrics:
        plt.subplot(4, n_metrics, plot_idx)
        sns.scatterplot(data=analysis_df, x=centrality, y=metric, alpha=0.5)
        plt.title(f"{centrality.replace('_', ' ').title()} vs {metric.replace('_', ' ').title()}")
        plt.xlabel(centrality.replace('_', ' ').title())
        plt.ylabel(metric.replace('_', ' ').title())
        plot_idx += 1
plt.tight_layout()
plt.savefig('./task5_scatter_plots.png')
print("Scatter plots saved to /task5_scatter_plots.png")

# Step 10: Visualize correlation matrix (Heatmap)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix: Centrality vs Metrics')
plt.savefig('./task5_heatmap.png')
print("Correlation heatmap saved to /task5_heatmap.png")

# Step 11: Save results
correlations.to_csv('./task5_correlations.csv')
analysis_df.to_csv('./task5_analysis.csv', index=False)
print("Saved correlations to /task5_correlations.csv and analysis to /task5_analysis.csv")

### 6. Task
## Community Detection (Unweighted)

- Apply community detection (e.g., Louvain or Girvan-Newman) on the unweighted network.
- Visualize and label resulting communities, analyze their shared features.


In [112]:

G_weighted = nx.read_gml('/kaggle/input/sna-project-files/task2_graph.gml')
df = pd.read_parquet('/kaggle/input/sna-project-files/task2_df.parquet')
print(f"Loaded weighted graph: {G_weighted.number_of_nodes()} nodes, {G_weighted.number_of_edges()} edges")
print(f"Loaded DataFrame: {len(df)} rows")


Loaded weighted graph: 4846 nodes, 11238 edges
Loaded DataFrame: 5000 rows


### Community Louvain Common Function for Task 6 and 7

In [113]:
def communityLouvainFuc(G, weight=False):
    
    # communityw = "community_weighted" if weight==True else "community_unweighted"
    
    if weight==False:
        partition = community_louvain.best_partition(G, random_state=42)
    else:
        partition = community_louvain.best_partition(G, weight='weight', random_state=42)
        
    print(f"Detected {len(set(partition.values()))} communities")
    
    # Create DataFrame with community assignments
    community_df = pd.DataFrame({
        'restaurant_name': list(partition.keys()),
        "community_weighted" if weight==True else "community_unweighted" : list(partition.values())
    })
    
    # Merge with original DataFrame for attributes
    community_df = community_df.merge(
        df[['restaurant_name', 'city', 'cuisines', 'special_diets', 'features', 'latitude', 'longitude']],
        on='restaurant_name',
        how='left'
    )

    return community_df




In [114]:
# def safe_literal_eval(val):
#     if isinstance(val, str):
#         try:
#             return ast.literal_eval(val)
#         except (ValueError, SyntaxError):
#             return []  
#     return val

# def parse_list_column(col):
#     return col.apply(safe_literal_eval)

# df['cuisines'] = parse_list_column(df['cuisines'])
# df['special_diets'] = parse_list_column(df['special_diets'])
# df['features'] = parse_list_column(df['features'])

# print(df['cuisines'])


# Step 1: Convert to unweighted graph
G = G_weighted.copy()
for u, v in G.edges():
    G[u][v].pop('weight', None)
print(f"Unweighted graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")


print(f"Connected components: {len(list(nx.connected_components(G)))}")
isolated = sum(1 for node in G.nodes() if G.degree(node) == 0)
print(f"Isolated nodes: {isolated}")


# Step 2: Apply Louvain community detection on weighted graph

community_unweighted= communityLouvainFuc(G)



Unweighted graph: 4846 nodes, 11238 edges
Connected components: 2961
Isolated nodes: 2417
Detected 2972 communities


In [115]:
# Step 3: Analyze communities
community_unweighted_sizes = community_unweighted['community_unweighted'].value_counts()
print("\nTop 5 Communities by Size:")
print(community_unweighted_sizes.head())

# Analyze shared features for top 3 communities
cuisines=[]
cities=[]
special_diets=[]
features=[]


# Safely parse comma-separated cuisines into a list
def extract_cuisine_items(column):
    all_cuisines = []
    for item in column:
        if isinstance(item, str):
            if item.strip() != '[]':  # exclude empty lists
                all_cuisines.extend([c.strip() for c in item.split(',')])
        elif isinstance(item, list):  # already parsed
            all_cuisines.extend(item)
    return all_cuisines

# Analysis block

top_communities = community_unweighted_sizes.head(10).index
print("\nShared Features of Top 3 Communities:")

for comm in top_communities:
    comm_df = community_unweighted[community_unweighted['community_unweighted'] == comm]
    print(f"\nCommunity {comm} ({len(comm_df)} restaurants):")

    cities += extract_cuisine_items(comm_df['city'])
    print(f"\nTop 5 cities: {Counter(cities).most_common(5)}")
    
    cuisines += extract_cuisine_items(comm_df['cuisines'])
    
    print(f"\nTop 5 cuisines: {Counter(cuisines).most_common(5)}")
    
    special_diets += extract_cuisine_items(comm_df['special_diets'])
    print(f"\nTop 5 special_diets: {Counter(special_diets).most_common(5)}")

    features += extract_cuisine_items(comm_df['features'])
    print(f"\nTop 5 features: {Counter(features).most_common(5)}")


Top 5 Communities by Size:
community_unweighted
26     156
201    102
2       98
44      97
217     60
Name: count, dtype: int64

Shared Features of Top 3 Communities:

Community 26 (156 restaurants):

Top 5 cities: [('Unknown', 113), ('Helsinki', 4), ('Dresden', 4), ('Dinslaken', 3), ('Weil am Rhein', 2)]

Top 5 cuisines: [('European', 25), ('Fast food', 23), ('British', 23), ('Mediterranean', 18), ('Italian', 16)]

Top 5 special_diets: [('Vegetarian Friendly', 49), ('Vegan Options', 31), ('Gluten Free Options', 23), ('Halal', 5)]

Top 5 features: [('Seating', 34), ('Wheelchair Accessible', 24), ('Takeout', 23), ('Reservations', 20), ('Table Service', 17)]

Community 201 (102 restaurants):

Top 5 cities: [('Unknown', 140), ('Frankfurt', 13), ('Dusseldorf', 9), ('Hannover', 5), ('Sheffield', 5)]

Top 5 cuisines: [('European', 39), ('Fast food', 37), ('British', 34), ('Cafe', 29), ('Mediterranean', 23)]

Top 5 special_diets: [('Vegetarian Friendly', 80), ('Vegan Options', 48), ('Gluten

In [116]:

G_top = G.subgraph(top_communities)
print(f"\nTop communities subgraph: {G_top.number_of_nodes()} nodes, {G_top.number_of_edges()} edges")

net = Network(notebook=True, height="600px", width="100%", directed=False, cdn_resources='in_line')
net.from_nx(G_top)

net = Network(notebook=True, height="600px", width="100%", directed=False, cdn_resources='in_line')

# Add nodes with colors and titles
colors = ['#FF9999', '#66CC99', '#99CCFF', '#FFCC99', '#CC99FF']
for node in G_top.nodes():
    comm = partition[node]
    city = community_unweighted[community_unweighted['restaurant_name'] == node]['city'].iloc[0]
    net.add_node(node, label=node, title=f"{node}\nCommunity: {comm}\nCity: {city}", color=colors[comm % len(colors)])

# Add edges with color
for source, target in G_top.edges():
    net.add_edge(source, target, color='#888888')

net.show_buttons(filter_=['physics'])
net.show("/kaggle/working/task6_community_graph.html")
print("Community visualization saved to /kaggle/working/task6_community_graph.html")



Top communities subgraph: 0 nodes, 0 edges
/kaggle/working/task6_community_graph.html
Community visualization saved to /kaggle/working/task6_community_graph.html


In [136]:

from IPython.display import IFrame
def showMyHtmlGraph(path):
    display(IFrame(path, width="100%", height="600px"))



In [137]:
# # Step 6: Save results

# display html graph
showMyHtmlGraph('task6_community_graph.html')

community_unweighted.to_csv('/kaggle/working/task6_communities.csv', index=False)
nx.write_gml(G, '/kaggle/working/task6_unweighted_graph.gml')
print("Saved community assignments to /kaggle/working/task6_communities.csv and unweighted graph to /kaggle/working/task6_unweighted_graph.gml")

Saved community assignments to /kaggle/working/task6_communities.csv and unweighted graph to /kaggle/working/task6_unweighted_graph.gml


### 7. Task
## Community Detection (Weighted)

- Run the same algorithm on the weighted network.
- Compare community structure between the two versions.

In [119]:
G_weighted = nx.read_gml('/kaggle/input/sna-project-files/task2_graph.gml')
df = pd.read_parquet('/kaggle/input/sna-project-files/task2_df.parquet')
print(f"Loaded weighted graph: {G_weighted.number_of_nodes()} nodes, {G_weighted.number_of_edges()} edges")
print(f"Loaded DataFrame: {len(df)} rows")


Loaded weighted graph: 4846 nodes, 11238 edges
Loaded DataFrame: 5000 rows


In [120]:

#  Apply Louvain community detection on weighted graph loaded from task 2

community_df_weighted= communityLouvainFuc(G, weight=True)


Detected 2972 communities


In [121]:
community_sizes_weighted = community_df_weighted['community_weighted'].value_counts()
print("\nTop 5 Weighted Communities by Size:")
print(community_sizes_weighted.head())


# Analyze shared features for top 3 communities
cuisines=[]
cities=[]
special_diets=[]
features=[]


top_communities = community_sizes_weighted.head(10).index
print("\nShared Features of Top 3 Communities:")

for comm in top_communities:
    comm_df = community_df_weighted[community_df_weighted['community_weighted'] == comm]
    print(f"\nCommunity {comm} ({len(comm_df)} restaurants):")

    cities += extract_cuisine_items(comm_df['city'])
    print(f"\nTop 5 cities: {Counter(cities).most_common(5)}")
    
    cuisines += extract_cuisine_items(comm_df['cuisines'])
    
    print(f"\nTop 5 cuisines: {Counter(cuisines).most_common(5)}")
    
    special_diets += extract_cuisine_items(comm_df['special_diets'])
    print(f"\nTop 5 special_diets: {Counter(special_diets).most_common(5)}")

    features += extract_cuisine_items(comm_df['features'])
    print(f"\nTop 5 features: {Counter(features).most_common(5)}")




Top 5 Weighted Communities by Size:
community_weighted
26     156
201    102
2       98
44      97
217     60
Name: count, dtype: int64

Shared Features of Top 3 Communities:

Community 26 (156 restaurants):

Top 5 cities: [('Unknown', 113), ('Helsinki', 4), ('Dresden', 4), ('Dinslaken', 3), ('Weil am Rhein', 2)]

Top 5 cuisines: [('European', 25), ('Fast food', 23), ('British', 23), ('Mediterranean', 18), ('Italian', 16)]

Top 5 special_diets: [('Vegetarian Friendly', 49), ('Vegan Options', 31), ('Gluten Free Options', 23), ('Halal', 5)]

Top 5 features: [('Seating', 34), ('Wheelchair Accessible', 24), ('Takeout', 23), ('Reservations', 20), ('Table Service', 17)]

Community 201 (102 restaurants):

Top 5 cities: [('Unknown', 140), ('Frankfurt', 13), ('Dusseldorf', 9), ('Hannover', 5), ('Sheffield', 5)]

Top 5 cuisines: [('European', 39), ('Fast food', 37), ('British', 34), ('Cafe', 29), ('Mediterranean', 23)]

Top 5 special_diets: [('Vegetarian Friendly', 80), ('Vegan Options', 48), (

### Comparision between weighted and unweighted graphs

In [122]:
from sklearn.metrics import adjusted_rand_score

# Step 3: Compare unweighted (Task 6) and weighted (Task 7) communities
community_df_unweighted = community_unweighted[['restaurant_name', 'community_unweighted']]
community_sizes_unweighted = community_df_unweighted['community_unweighted'].value_counts()
print("\nComparison of Community Structures:")
print(f"Unweighted (Task 6): {len(set(community_df_unweighted['community_unweighted']))} communities")
print(f"Weighted (Task 7): {len(set(community_df_weighted['community_weighted']))} communities")

print("\nTop 5 Unweighted Community Sizes:")
print(community_sizes_unweighted.head())
print("\nTop 5 Weighted Community Sizes:")
print(community_sizes_weighted.head())

comparison_df = community_unweighted.merge(
    community_df_weighted[['restaurant_name', 'community_weighted']],
    on='restaurant_name',
    how='inner'
)
ari = adjusted_rand_score(community_unweighted['community_unweighted'], community_df_weighted['community_weighted'])
print(f"\nAdjusted Rand Index (ARI): {ari:.4f}")
print("(ARI close to 1: similar partitions; close to 0: dissimilar)")


from collections import Counter

comparison_cities = []
comparison_cuisines = []

def extract_cuisine_items(column):
    all_cuisines = []
    for item in column:
        if isinstance(item, str) and item.strip() != '[]':
            all_cuisines.extend([c.strip() for c in item.split(',')])
    return all_cuisines


top_communities_unweighted = community_sizes_unweighted.head(10).index
top_communities_weighted = community_sizes_weighted.head(10).index


# Helper function to compute Jaccard similarity for sets (e.g., cuisines)
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

# Compare shared features of top 3 communities
print("\n=== Shared Features Comparison (Top 3 Communities) ===")
for comm_type, comm_df, top_comms, prefix in [
    ('Unweighted', community_unweighted, top_communities_unweighted, 'community_unweighted'),
    ('Weighted', community_df_weighted, top_communities_weighted, 'community_weighted')
]:
    print(f"\n--- {comm_type} Top 5 Communities ---")
    for comm in top_comms:
        comm_data = comm_df[comm_df[prefix] == comm]
        print(f"\nCommunity {comm} ({len(comm_data)} restaurants):")
        cities = Counter(comm_data['city'].dropna())
        print(f"  Top Cities: {cities.most_common(10)}")
        cuisines = extract_cuisine_items(comm_data['cuisines'])
        cuisine_counts = Counter(cuisines)
        print(f"  Top Cuisines: {cuisine_counts.most_common(10)}")
        # print(f"  Top Cuisines: {cuisines.most_common(5)}")
        
        # special_diets = Counter([sd for sublist in comm_data['special_diets'].dropna() for sd in sublist])
        special_diets = extract_cuisine_items(comm_data['special_diets'])
        special_diets_counts = Counter(special_diets)
        print(f"  Top Special Diets: {special_diets_counts.most_common(10)}")
        # features = Counter([f for sublist in comm_data['features'].dropna() for f in sublist])
        features = extract_cuisine_items(comm_data['features'])
        features_counts = Counter(features)
        print(f"  Top Features: {features_counts.most_common(10)}")







Comparison of Community Structures:
Unweighted (Task 6): 2972 communities
Weighted (Task 7): 2972 communities

Top 5 Unweighted Community Sizes:
community_unweighted
26     156
201    102
2       98
44      97
217     60
Name: count, dtype: int64

Top 5 Weighted Community Sizes:
community_weighted
26     156
201    102
2       98
44      97
217     60
Name: count, dtype: int64

Adjusted Rand Index (ARI): 1.0000
(ARI close to 1: similar partitions; close to 0: dissimilar)

=== Shared Features Comparison (Top 3 Communities) ===

--- Unweighted Top 5 Communities ---

Community 26 (156 restaurants):
  Top Cities: [('Unknown', 113), ('Helsinki', 4), ('Dresden', 4), ('Dinslaken', 3), ('Weil am Rhein', 2), ('Llanelli', 2), ('Backnang', 2), ('Wuppertal', 2), ('Delmenhorst', 2), ('Nuremberg', 2)]
  Top Cuisines: [('European', 25), ('Fast food', 23), ('British', 23), ('Mediterranean', 18), ('Italian', 16), ('Cafe', 13), ('Pub', 10), ('American', 9), ('Asian', 9), ('Middle Eastern', 8)]
  Top Sp

In [123]:
def extract_cuisine_set(series):
    cuisines = set()
    for item in series.dropna():
        if isinstance(item, str):
            cuisines.update([c.strip() for c in item.split(',')])
        elif isinstance(item, list):
            cuisines.update(item)
    return cuisines


# Compare cuisine overlap between top communities
print("\n=== Cuisine Overlap (Jaccard Similarity) ===\n\n")

for i, (comm_unw, comm_w) in enumerate(zip(top_communities[:1], top_communities_weighted[:1])):
    unw_data = community_unweighted[community_unweighted['community_unweighted'] == comm_unw]
    w_data = community_df_weighted[community_df_weighted['community_weighted'] == comm_w]
    # print("Unweighted Data Columns:", unw_data.columns.tolist())
    # print("Weighted Data Columns:", w_data.columns.tolist())
    # print("Unweighted Data Columns: Cuisines -> ", w_data['cuisines'].head())
    # print("Weighted Data Columns: Cuisines -> ", w_data['cuisines'].head())

    # unw_cuisines = set(c for sublist in unw_data['cuisines'].dropna())
    # w_cuisines = set(c for sublist in w_data['cuisines'].dropna() for c in sublist)
    unw_cuisines = extract_cuisine_set(unw_data['cuisines'])
    w_cuisines = extract_cuisine_set(w_data['cuisines'])
    jaccard = jaccard_similarity(unw_cuisines, w_cuisines)
    print(f"Top Community {i+1} (Unweighted {comm_unw} vs. Weighted {comm_w}): Jaccard Similarity = {jaccard:.4f}")





=== Cuisine Overlap (Jaccard Similarity) ===


Top Community 1 (Unweighted 26 vs. Weighted 26): Jaccard Similarity = 1.0000


In [124]:


# Add nodes with colors and titles
colors = ['#FF9999', '#66CC99', '#99CCFF', '#FFCC99', '#CC99FF']
for node in G_top.nodes():
    comm = partition[node]
    city = community_df_weighted[community_df_weighted['restaurant_name'] == node]['city'].iloc[0]
    net.add_node(node, label=node, title=f"{node}\nCommunity: {comm}\nCity: {city}", color=colors[comm % len(colors)])

# Add edges with color
for source, target in G_top.edges():
    net.add_edge(source, target, color='#888888')



net.show_buttons(filter_=['physics'])
net.show("/kaggle/working/task7_community_weighted_graph.html")
print("Community visualization saved to /kaggle/working/task7_community_weighted_graph.html")

# net.show("/kaggle/working/task6_community_graph.html")  # This creates the file
# display(IFrame("task7_community_graph.html", width="100%", height="600px"))
showMyHtmlGraph('task7_community_weighted_graph.html')



/kaggle/working/task7_community_weighted_graph.html
Community visualization saved to /kaggle/working/task7_community_weighted_graph.html


In [125]:
# Step 5: Save results
community_df_weighted.to_csv('/kaggle/working/task7_communities.csv', index=False)
nx.write_gml(G, '/kaggle/working/task7_weighted_graph.gml')
print("Saved community assignments to /kaggle/working/task7_communities.csv and unweighted graph to /kaggle/working/task7_weighted_graph.gml")

Saved community assignments to /kaggle/working/task7_communities.csv and unweighted graph to /kaggle/working/task7_weighted_graph.gml


### 8. Role of Dietary Preferences

- Filter nodes with vegetarian_friendly, vegan_options, or gluten_free tags.
- Analyze their position in the network (e.g., density, centrality, community inclusion).

In [126]:

# Load Task 2 weighted graph, DataFrame, and Task 7 communities
G = nx.read_gml('/kaggle/input/sna-project-files/task2_graph.gml')
df = pd.read_parquet('/kaggle/input/sna-project-files/task2_df.parquet')
community_df_weighted = pd.read_csv('/kaggle/working/task7_communities.csv')
print(f"Loaded weighted graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Loaded DataFrame: {len(df)} rows")
print(f"Loaded Task 7 communities: {len(community_df_weighted)} restaurants")


Loaded weighted graph: 4846 nodes, 11238 edges
Loaded DataFrame: 5000 rows
Loaded Task 7 communities: 5000 restaurants




### Step 1: Filter nodes with vegetarian_friendly, vegan_options, or gluten_free tags
### Parse special_diets column (stringified lists)

In [127]:

print(df.columns.tolist())

 # Define dietary tags

dietary_tags = ['vegetarian_friendly', 'vegan_options', 'gluten_free']
dietary_nodes = []

# Loop through each dietary tag column
for tag in dietary_tags:
    if tag in df.columns:
        matches = df[df[tag] == 'Y']['restaurant_name'].tolist()
        dietary_nodes.extend(matches)

# Remove duplicates
dietary_nodes = list(set(dietary_nodes))
print(f"\n\n====== \n\n Found  {len(dietary_nodes)} restaurants with dietary tags\n\n======")


['restaurant_link', 'restaurant_name', 'original_location', 'country', 'region', 'province', 'city', 'address', 'latitude', 'longitude', 'claimed', 'awards', 'popularity_detailed', 'popularity_generic', 'top_tags', 'price_level', 'price_range', 'meals', 'cuisines', 'special_diets', 'features', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'original_open_hours', 'open_days_per_week', 'open_hours_per_week', 'working_shifts_per_week', 'avg_rating', 'total_reviews_count', 'default_language', 'reviews_count_in_default_language', 'excellent', 'very_good', 'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere', 'keywords']



 Found  1585 restaurants with dietary tags



In [128]:

# Step 2: Analyze network position
# Create subgraph of dietary nodes

G_dietary = G.subgraph(dietary_nodes)
print(f"Dietary subgraph: {G_dietary.number_of_nodes()} nodes, {G_dietary.number_of_edges()} edges")

# Density of dietary subgraph
density = nx.density(G_dietary)

full_graph_density = nx.density(G)
print(f"\nDensity Analysis:")
print(f"Dietary subgraph density: {density:.6f}")
print(f"Full graph density: {full_graph_density:.6f}")
print("(Higher density means more connections among dietary restaurants)")

# Centrality in the full graph
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, k=min(1000, G.number_of_nodes()), weight='weight')

# Compute centrality for dietary nodes
dietary_centrality = pd.DataFrame({
    'restaurant_name': dietary_nodes,
    'degree_centrality': [degree_centrality[node] for node in dietary_nodes],
    'betweenness_centrality': [betweenness_centrality[node] for node in dietary_nodes]
})


# Average centrality for dietary vs. all nodes
avg_degree_dietary = dietary_centrality['degree_centrality'].mean()
avg_degree_all = sum(degree_centrality.values()) / len(degree_centrality)
avg_betweenness_dietary = dietary_centrality['betweenness_centrality'].mean()
avg_betweenness_all = sum(betweenness_centrality.values()) / len(betweenness_centrality)

print(f"\nCentrality Analysis:")
print(f"Average degree centrality (dietary): {avg_degree_dietary:.6f}")
print(f"Average degree centrality (all): {avg_degree_all:.6f}")
print(f"Average betweenness centrality (dietary): {avg_betweenness_dietary:.6f}")
print(f"Average betweenness centrality (all): {avg_betweenness_all:.6f}")
print("(Higher centrality means dietary restaurants are more connected or influential)")


Dietary subgraph: 1585 nodes, 1251 edges

Density Analysis:
Dietary subgraph density: 0.000997
Full graph density: 0.000957
(Higher density means more connections among dietary restaurants)

Centrality Analysis:
Average degree centrality (dietary): 0.000972
Average degree centrality (all): 0.000957
Average betweenness centrality (dietary): 0.000013
Average betweenness centrality (all): 0.000011
(Higher centrality means dietary restaurants are more connected or influential)


In [129]:

# Top 5 dietary nodes by centrality
print(f"\nTop 5 Dietary Restaurants by Degree Centrality:")
print(dietary_centrality.sort_values('degree_centrality', ascending=False)[['restaurant_name', 'degree_centrality']].head())
print(f"\nTop 5 Dietary Restaurants by Betweenness Centrality:")
print(dietary_centrality.sort_values('betweenness_centrality', ascending=False)[['restaurant_name', 'betweenness_centrality']].head())

# Community inclusion (using Task 7 weighted communities)
dietary_communities = community_df_weighted[community_df_weighted['restaurant_name'].isin(dietary_nodes)]
community_distribution = dietary_communities['community_weighted'].value_counts()
print(f"\nCommunity Inclusion Analysis:")
print(f"Dietary restaurants in Task 7 communities (top 5):")
print(community_distribution.head())
print(f"Total communities with dietary restaurants: {len(community_distribution)}")

# Analyze dietary tags per community


# from collections import Counter

# Define dietary tags
# dietary_tags = ['vegetarian_friendly', 'vegan_options', 'gluten_free']

print("\n=== Dietary Tags in Top 10 Communities ===")
top_communities = community_df_weighted['community_weighted'].value_counts().head(10).index

for comm in top_communities:
    # Filter the community
    comm_df = community_df_weighted[community_df_weighted['community_weighted'] == comm]
    
    # Find all restaurant names in this community
    comm_nodes = comm_df['restaurant_name'].tolist()
    
    # Initialize tag counter
    tag_counter = Counter()
    
    # For each tag, check which restaurants in this community have 'Y'
    for tag in dietary_tags:
        # Get restaurant names where the tag is 'Y'
        y_nodes = df[df[tag] == 'Y']['restaurant_name']
        
        # Count how many of those nodes are in the current community
        count = sum(name in comm_nodes for name in y_nodes)
        tag_counter[tag] = count

    print(f"\nCommunity {comm} ({len(comm_df)} total restaurants):")
    for tag, count in tag_counter.items():
        print(f"  {tag}: {count} restaurants with '{tag}' = Y")




Top 5 Dietary Restaurants by Degree Centrality:
         restaurant_name  degree_centrality
241        Pret a Manger           0.016718
1428        Café Etienne           0.015067
494        Le Gribouille           0.014861
1295            Angelina           0.014654
757   Restaurant Sannine           0.014448

Top 5 Dietary Restaurants by Betweenness Centrality:
           restaurant_name  betweenness_centrality
241          Pret a Manger                0.003575
1570  Bread Street Kitchen                0.003275
503         Domino's Pizza                0.002543
1524          Ugarit Sants                0.001853
1445         Mudec Bistrot                0.001353

Community Inclusion Analysis:
Dietary restaurants in Task 7 communities (top 5):
community_weighted
26     51
201    31
2      27
44     25
217    23
Name: count, dtype: int64
Total communities with dietary restaurants: 1133

=== Dietary Tags in Top 10 Communities ===

Community 26 (156 total restaurants):
  vegetarian_frien

In [132]:
# Step 3: Visualize dietary nodes
def visualize_dietary_nodes(graph, nodes, df, community_df, output_file):
    """Visualize a sample of dietary nodes with PyVis."""
   
    G_sample = graph.subgraph(nodes)
    print(f"Visualization subgraph: {G_sample.number_of_nodes()} nodes, {G_sample.number_of_edges()} edges")
    
    net = Network(notebook=True, height="600px", width="100%", directed=False, cdn_resources='in_line')
    
    colors = {
        'vegetarian_friendly': '#66CC99',
        'vegan_options': '#FF9999',
        'gluten_free': '#99CCFF',
        'multiple': '#CC99FF',
        'unknown': '#888888'
    }
    
    # Set node properties
    for node in G_sample.nodes():
        node_row = df[df['restaurant_name'] == node]
        if not node_row.empty:
            node_tags = [tag for tag in node_row['special_diets'].iloc[0] if tag in colors]
            color = colors['multiple'] if len(node_tags) > 1 else colors.get(node_tags[0] if node_tags else 'unknown')
        else:
            node_tags = ['unknown']
            color = colors['unknown']
        
        comm_row = community_df[community_df['restaurant_name'] == node]
        comm = comm_row['community_weighted'].iloc[0] if not comm_row.empty else 'unknown'
        
        net.add_node(
            node,
            label=node,
            color=color,
            title=f"{node}\nTags: {node_tags}\nCommunity: {comm}"
        )
    
    # Set edge properties
    for edge in G_sample.edges(data=True):
        net.add_edge(
            edge[0],
            edge[1],
            color='#888888',
            value=edge[2].get('weight', 1),
            title=f"Weight: {edge[2].get('weight', 1):.2f}"
        )
    
    net.show_buttons(filter_=['physics'])
    net.show(output_file)
    print(f"Visualization saved to {output_file}")

visualize_dietary_nodes(G, dietary_nodes, df, community_df_weighted, "/kaggle/working/task8_dietary_graph.html")



Visualization subgraph: 1585 nodes, 1251 edges
/kaggle/working/task8_dietary_graph.html
Visualization saved to /kaggle/working/task8_dietary_graph.html


In [134]:
# Step 4: Save results
dietary_centrality['community_weighted'] = dietary_centrality['restaurant_name'].map(
    community_df_weighted.set_index('restaurant_name')['community_weighted'].get
)
# Check for missing community assignments
missing_communities = dietary_centrality['community_weighted'].isna().sum()
print(f"Missing community assignments: {missing_communities}")
dietary_centrality.to_csv('/kaggle/working/task8_dietary_analysis.csv', index=False)
print("Saved analysis to /kaggle/working/task8_dietary_analysis.csv")

Missing community assignments: 0
Saved analysis to /kaggle/working/task8_dietary_analysis.csv


In [139]:
from IPython.display import IFrame
showMyHtmlGraph('task8_dietary_graph.html')

# task9
## Subnetwork Analysis by Region 
- Extract subnetworks by city or country. 
- Compare SNA statistics: density, diameter, clustering coefficient.

In [3]:
!pip install pyvis seaborn


[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# Load Task 2 graph and DataFrame
G = nx.read_gml('./task2_graph.gml')
df = pd.read_parquet('./task2_df.parquet')
print(f"Loaded graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Loaded DataFrame: {len(df)} rows")

# Step 1: Extract subnetworks by city
# Check available columns
print("DataFrame columns:", df.columns.tolist())

# Verify city column exists
if 'city' not in df.columns:
    raise KeyError("Column 'city' not found in task2_df.parquet. Please check the DataFrame.")

# Get top 10 cities by restaurant count
city_counts = df['city'].value_counts()
top_cities = city_counts.head(10).index.tolist()
print(f"Top 10 cities by restaurant count:\n{city_counts.head(10)}")

# Create subnetworks for top cities
subgraphs = {}
for city in top_cities:
    # Get restaurant names in this city
    city_nodes = df[df['city'] == city]['restaurant_name'].tolist()
    # Filter nodes that exist in the graph
    city_nodes = [node for node in city_nodes if node in G.nodes()]
    # Create subgraph
    subgraph = G.subgraph(city_nodes)
    subgraphs[city] = subgraph
    print(f"Subgraph for {city}: {subgraph.number_of_nodes()} nodes, {subgraph.number_of_edges()} edges")

# Step 2: Compute SNA statistics
sna_stats = []
for city, subgraph in subgraphs.items():
    if subgraph.number_of_nodes() == 0:
        continue
    # Density
    density = nx.density(subgraph) if subgraph.number_of_nodes() > 1 else 0
    # Diameter (use largest connected component)
    try:
        components = list(nx.connected_components(subgraph))
        if components:
            largest_cc = max(components, key=len)
            subgraph_cc = subgraph.subgraph(largest_cc)
            diameter = nx.diameter(subgraph_cc) if subgraph_cc.number_of_nodes() > 1 else 0
        else:
            diameter = 0
    except:
        diameter = 0
    # Clustering coefficient (weighted)
    clustering = nx.average_clustering(subgraph, weight='weight') if subgraph.number_of_nodes() > 1 else 0
    sna_stats.append({
        'city': city,
        'nodes': subgraph.number_of_nodes(),
        'edges': subgraph.number_of_edges(),
        'density': density,
        'diameter': diameter,
        'clustering_coefficient': clustering
    })

# Create DataFrame for SNA stats
sna_df = pd.DataFrame(sna_stats)
print("\nSNA Statistics for Top Cities:")
print(sna_df)

# Step 3: Compare SNA statistics
# Plot comparison
plt.figure(figsize=(12, 8))
sns.barplot(data=sna_df.melt(id_vars=['city'], value_vars=['density', 'diameter', 'clustering_coefficient']),
            x='city', y='value', hue='variable')
plt.title('SNA Statistics Comparison Across Top Cities')
plt.xticks(rotation=45)
plt.ylabel('Value')
plt.tight_layout()
plt.savefig('./task9_sna_comparison.png')
plt.close()
print("Saved SNA comparison plot to /task9_sna_comparison.png")

# Step 4: Visualize one city's subnetwork (e.g., top city)
def visualize_subnetwork(graph, city, df, output_file, sample_size=300):
    """Visualize a city's subnetwork with PyVis."""
    sample_nodes = list(graph.nodes())[:min(sample_size, graph.number_of_nodes())]
    G_sample = graph.subgraph(sample_nodes)
    print(f"Visualization subgraph for {city}: {G_sample.number_of_nodes()} nodes, {G_sample.number_of_edges()} edges")
    
    net = Network(notebook=True, height="600px", width="100%", directed=False, cdn_resources='in_line')
    
    # Set node properties
    for node in G_sample.nodes():
        node_row = df[df['restaurant_name'] == node]
        color = '#66CC99' if not node_row.empty else '#888888'
        net.add_node(
            node,
            label=node,
            color=color,
            title=f"{node}\nCity: {city}"
        )
    
    # Set edge properties
    for edge in G_sample.edges(data=True):
        net.add_edge(
            edge[0],
            edge[1],
            color='#888888',
            value=edge[2].get('weight', 1),
            title=f"Weight: {edge[2].get('weight', 1):.2f}"
        )
    
    net.show_buttons(filter_=['physics'])
    net.show(output_file)
    print(f"Visualization saved to {output_file}")

# Visualize the top city
top_city = top_cities[0]
visualize_subnetwork(subgraphs[top_city], top_city, df, f"./task9_{top_city}_graph.html")

# Step 5: Save SNA stats
sna_df.to_csv('./task9_sna_stats.csv', index=False)
print("Saved SNA statistics to ./task9_sna_stats.csv")

Loaded graph: 4846 nodes, 11238 edges
Loaded DataFrame: 5000 rows
DataFrame columns: ['restaurant_link', 'restaurant_name', 'original_location', 'country', 'region', 'province', 'city', 'address', 'latitude', 'longitude', 'claimed', 'awards', 'popularity_detailed', 'popularity_generic', 'top_tags', 'price_level', 'price_range', 'meals', 'cuisines', 'special_diets', 'features', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'original_open_hours', 'open_days_per_week', 'open_hours_per_week', 'working_shifts_per_week', 'avg_rating', 'total_reviews_count', 'default_language', 'reviews_count_in_default_language', 'excellent', 'very_good', 'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere', 'keywords']
Top 10 cities by restaurant count:
city
Unknown      1831
Paris          79
Madrid         53
Rome           49
Milan          45
Prague         29
Vienna         23
Amsterdam      22
Lisbon         21
Munich         18
Name: count, dtype: int64
Subgraph for Unknow

# task 10
## Visualization of the Global Network 
- Visualize the full network with: 
- Node color = community 
- Node size = total reviews or rating 
- Edge width = weight

In [15]:
import matplotlib.colors as mcolors# Load Task 2 graph and DataFrame, and Task 7 communities
G = nx.read_gml('./task2_graph.gml')
df = pd.read_parquet('./task2_df.parquet')
community_df = pd.read_csv('./task7_communities.csv')
print(f"Loaded graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Loaded DataFrame: {len(df)} rows")
print(f"Loaded communities: {len(community_df)} restaurants")

# Verify required columns
print("DataFrame columns:", df.columns.tolist())
print("Community DataFrame columns:", community_df.columns.tolist())
if 'total_reviews' not in df.columns:
    print("Warning: 'total_reviews' not found. Using uniform node sizes.")
if 'community_weighted' not in community_df.columns:
    raise KeyError("Column 'community_weighted' not found in task7_communities.csv.")

# Step 1: Merge community data with DataFrame
df = df.merge(community_df[['restaurant_name', 'community_weighted']], on='restaurant_name', how='left')
print(f"Merged DataFrame: {len(df)} rows")

# Step 2: Sample nodes proportionally by community
sample_size = 1000
community_counts = df['community_weighted'].value_counts()
total_nodes = len(df)
sample_proportions = community_counts / total_nodes
sample_sizes = (sample_proportions * sample_size).round().astype(int)
sample_sizes = sample_sizes[sample_sizes > 0]

sampled_nodes = []
for comm, size in sample_sizes.items():
    comm_nodes = df[df['community_weighted'] == comm]['restaurant_name'].tolist()
    comm_nodes = [node for node in comm_nodes if node in G.nodes()]
    sampled = np.random.choice(comm_nodes, size=min(size, len(comm_nodes)), replace=False)
    sampled_nodes.extend(sampled)

# Ensure sample size is met
if len(sampled_nodes) < sample_size:
    remaining = sample_size - len(sampled_nodes)
    other_nodes = [n for n in G.nodes() if n not in sampled_nodes]
    sampled_nodes.extend(np.random.choice(other_nodes, size=min(remaining, len(other_nodes)), replace=False))

sampled_nodes = sampled_nodes[:sample_size]
print(f"Sampled {len(sampled_nodes)} nodes for visualization")

# Step 3: Create sampled subgraph
G_sample = G.subgraph(sampled_nodes)
print(f"Sampled subgraph: {G_sample.number_of_nodes()} nodes, {G_sample.number_of_edges()} edges")

# Step 4: Visualize the global network
def visualize_global_network(graph, df, output_file, sample_size=1000):
    """Visualize the sampled global network with PyVis."""
    net = Network(notebook=True, height="800px", width="100%", directed=False, cdn_resources='in_line')
    
    # Define color map for communities
    unique_communities = df['community_weighted'].dropna().unique()
    n_communities = len(unique_communities)
    print(f"Number of unique communities: {n_communities}")
    color_map = plt.colormaps['viridis']  # Updated API for Matplotlib >= 3.7.0
    comm_to_color = {comm: mcolors.rgb2hex(color_map(i / n_communities)) for i, comm in enumerate(unique_communities)}
    
    # Normalize total_reviews for node sizes (between 10 and 50)
    if 'total_reviews_count' in df.columns:
        reviews = df['total_reviews_count'].dropna()
        min_reviews, max_reviews = reviews.min(), reviews.max()
        if max_reviews > min_reviews:
            size_scaling = lambda x: 10 + 40 * (x - min_reviews) / (max_reviews - min_reviews)
        else:
            size_scaling = lambda x: 20
    else:
        size_scaling = lambda x: 20
    
    # Set node properties
    for node in graph.nodes():
        node_row = df[df['restaurant_name'] == node]
        if not node_row.empty:
            comm = node_row['community_weighted'].iloc[0] if pd.notna(node_row['community_weighted'].iloc[0]) else 'unknown'
            color = comm_to_color.get(comm, '#888888')
            size = size_scaling(node_row['total_reviews_count'].iloc[0]) if 'total_reviews_count' in node_row.columns and pd.notna(node_row['total_reviews_count'].iloc[0]) else 20
            title = f"Restaurant: {node}\nCommunity: {comm}\nReviews: {node_row['total_reviews_count'].iloc[0] if 'total_reviews_count' in node_row.columns else 'N/A'}"
        else:
            color = '#888888'
            size = 20
            title = f"Restaurant: {node}\nCommunity: unknown\nReviews: N/A"
        
        net.add_node(
            node,
            label=node,
            color=color,
            size=size,
            title=title
        )
    
    # Set edge properties
    for edge in graph.edges(data=True):
        weight = edge[2].get('weight', 1)
        net.add_edge(
            edge[0],
            edge[1],
            value=weight * 2,  # Scale for visibility
            title=f"Weight: {weight:.2f}",
            color='#888888'
        )
    
    net.show_buttons(filter_=['physics'])
    net.show(output_file)
    print(f"Visualization saved to {output_file}")

visualize_global_network(G_sample, df, "./task10_global_network.html")

# Step 5: Save sampled nodes for reference
sampled_df = df[df['restaurant_name'].isin(sampled_nodes)][['restaurant_name', 'community_weighted', 'total_reviews_count']]
sampled_df.to_csv('./task10_sampled_nodes.csv', index=False)
print("Saved sampled nodes to ./task10_sampled_nodes.csv")

Loaded graph: 4846 nodes, 11238 edges
Loaded DataFrame: 5000 rows
Loaded communities: 5000 restaurants
DataFrame columns: ['restaurant_link', 'restaurant_name', 'original_location', 'country', 'region', 'province', 'city', 'address', 'latitude', 'longitude', 'claimed', 'awards', 'popularity_detailed', 'popularity_generic', 'top_tags', 'price_level', 'price_range', 'meals', 'cuisines', 'special_diets', 'features', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'original_open_hours', 'open_days_per_week', 'open_hours_per_week', 'working_shifts_per_week', 'avg_rating', 'total_reviews_count', 'default_language', 'reviews_count_in_default_language', 'excellent', 'very_good', 'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere', 'keywords']
Community DataFrame columns: ['restaurant_name', 'community_weighted', 'city', 'cuisines', 'special_diets', 'features', 'latitude', 'longitude']
Merged DataFrame: 7098 rows
Sampled 1000 nodes for visualization
Sampled subgraph: 

In [17]:
from IPython.display import IFrame
def showMyHtmlGraph(path):
    display(IFrame(path, width="100%", height="600px"))
showMyHtmlGraph("./task10_global_network.html")

# task 11
## Keyword Similarity Network 
- Build a separate network using similarity between keywords (e.g., using TF-IDF or cosine similarity). 
- Link restaurants with similar descriptions and analyze communities in this network. 

In [1]:
!pip install pyvis python-louvain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.0 MB/s eta 0:00:00
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9389 sha256=241cc1d8f7d093350bb9f66721d876a582b1a0a23f27d3230492fc09bf703f9a
  Stored in directory: /home/jovyan/.cache/pip/wheels/11/c1/e7/f62a211c636275e2da798bf0c307a3ae79aeddaf2524a03ce4
Successfully built python-louvain

[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from community import community_louvain



df = pd.read_parquet('./task2_df.parquet')
print(f"Loaded DataFrame: {len(df)} rows")

# Verify columns
print("DataFrame columns:", df.columns.tolist())

# Step 1: Prepare text data
# Use 'special_diets' and 'cuisines' for richer text
required_columns = ['restaurant_name', 'special_diets', 'cuisines']
for col in required_columns:
    if col not in df.columns:
        raise KeyError(f"Column '{col}' not found in task2_df.parquet.")

# Sample 5,000 restaurants (full dataset in this case)
sample_size = 5000
if len(df) > sample_size:
    df_sample = df.sample(n=sample_size, random_state=42)
else:
    df_sample = df
df_sample = df_sample.reset_index(drop=True)  # Ensure sequential indices
print(f"Sampled DataFrame: {len(df_sample)} rows")

# Clean and preprocess text
def preprocess_text(text):
    if pd.isna(text):
        return ""
    if isinstance(text, list):
        text = ", ".join(text)
    return str(text).lower()

df_sample['text'] = df_sample[['special_diets', 'cuisines']].apply(
    lambda x: ' '.join(x.dropna().astype(str)), axis=1
).apply(preprocess_text)
print("Preprocessed text data")

# Step 2: Compute TF-IDF and cosine similarity
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_sample['text'])
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix)
print(f"Cosine similarity matrix shape: {cosine_sim.shape}")

# Step 3: Build similarity network
G = nx.Graph()
similarity_threshold = 0.3  # Reduced to balance edge density

# Add nodes
for idx, row in df_sample.iterrows():
    G.add_node(row['restaurant_name'])

# Add edges
for i in range(len(cosine_sim)):
    for j in range(i + 1, len(cosine_sim)):
        sim_score = cosine_sim[i, j]
        if sim_score > similarity_threshold:
            restaurant_i = df_sample.iloc[i]['restaurant_name']
            restaurant_j = df_sample.iloc[j]['restaurant_name']
            G.add_edge(restaurant_i, restaurant_j, weight=sim_score)

print(f"Similarity network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Step 4: Detect communities
partition = community_louvain.best_partition(G, weight='weight', resolution=0.8, random_state=42)
communities = {}
for node, comm_id in partition.items():
    if comm_id not in communities:
        communities[comm_id] = []
    communities[comm_id].append(node)

print(f"Detected {len(communities)} communities")

# Step 5: Analyze communities
# Create community DataFrame
comm_df = pd.DataFrame({
    'restaurant_name': partition.keys(),
    'community': partition.values()
})

# Summarize community sizes
comm_sizes = pd.Series(partition.values()).value_counts().reset_index()
comm_sizes.columns = ['community', 'size']

# Extract top keywords per community
feature_names = vectorizer.get_feature_names_out()
community_keywords = []
for comm_id in communities:
    comm_nodes = communities[comm_id]
    # Map restaurant names to positional indices
    comm_indices = df_sample.index[df_sample['restaurant_name'].isin(comm_nodes)].tolist()
    if not comm_indices:
        continue  # Skip empty communities
    # Sum TF-IDF scores
    comm_tfidf = tfidf_matrix[comm_indices].sum(axis=0).A1
    # Get top 5 keywords
    top_indices = comm_tfidf.argsort()[-5:][::-1]
    top_keywords = [feature_names[idx] for idx in top_indices]
    community_keywords.append({
        'community': comm_id,
        'size': len(comm_nodes),
        'top_keywords': ", ".join(top_keywords)
    })

comm_summary = pd.DataFrame(community_keywords)
print("\nCommunity Summary:")
print(comm_summary)

# Step 6: Visualize a sampled subgraph
def visualize_similarity_network(graph, partition, df, output_file, sample_size=300):
    """Visualize a sampled subgraph with PyVis."""
    sample_nodes = list(graph.nodes())[:min(sample_size, graph.number_of_nodes())]
    G_sample = graph.subgraph(sample_nodes)
    print(f"Visualization subgraph: {G_sample.number_of_nodes()} nodes, {G_sample.number_of_edges()} edges")
    
    net = Network(notebook=True, height="600px", width="100%", directed=False, cdn_resources='in_line')
    
    unique_comms = set(partition.values())
    n_comms = len(unique_comms)
    color_map = plt.colormaps['viridis']
    comm_to_color = {comm: mcolors.rgb2hex(color_map(i / n_comms)) for i, comm in enumerate(unique_comms)}
    
    for node in G_sample.nodes():
        comm = partition.get(node, -1)
        color = comm_to_color.get(comm, '#888888')
        net.add_node(
            node,
            label=node,
            color=color,
            size=20,
            title=f"Restaurant: {node}\nCommunity: {comm}"
        )
    
    for edge in G_sample.edges(data=True):
        net.add_edge(
            edge[0],
            edge[1],
            value=edge[2].get('weight', 1) * 2,
            title=f"Similarity: {edge[2].get('weight', 1):.2f}",
            color='#888888'
        )
    
    net.show_buttons(filter_=['physics'])
    net.show(output_file)
    print(f"Visualization saved to {output_file}")

visualize_similarity_network(G, partition, df_sample, "./task11_similarity_network.html")

# Step 7: Save outputs
nx.write_gml(G, './task11_similarity_graph.gml')
comm_df.to_csv('./task11_communities.csv', index=False)
comm_summary.to_csv('./task11_community_summary.csv', index=False)
print("Saved similarity graph to ./task11_similarity_graph.gml")
print("Saved community assignments to ./task11_communities.csv")
print("Saved community summary to ./task11_community_summary.csv")

Loaded DataFrame: 5000 rows
DataFrame columns: ['restaurant_link', 'restaurant_name', 'original_location', 'country', 'region', 'province', 'city', 'address', 'latitude', 'longitude', 'claimed', 'awards', 'popularity_detailed', 'popularity_generic', 'top_tags', 'price_level', 'price_range', 'meals', 'cuisines', 'special_diets', 'features', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'original_open_hours', 'open_days_per_week', 'open_hours_per_week', 'working_shifts_per_week', 'avg_rating', 'total_reviews_count', 'default_language', 'reviews_count_in_default_language', 'excellent', 'very_good', 'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere', 'keywords']
Sampled DataFrame: 5000 rows
Preprocessed text data
TF-IDF matrix shape: (5000, 138)
Cosine similarity matrix shape: (5000, 5000)
Similarity network: 4846 nodes, 1228947 edges
Detected 676 communities

Community Summary:
     community  size                                       top_keywords
0         

In [13]:
from IPython.display import IFrame
def showMyHtmlGraph(path):
    display(IFrame(path, width="100%", height="600px"))
showMyHtmlGraph("./task11_similarity_network.html")

# task12
## Backbone Extraction 
- Use techniques like disparity filtering to extract the most informative subnetwork. 
- Analyze and visualize the resulting structure. 

In [5]:

import matplotlib.colors as mcolors



# Install required package
!pip install pyvis

# Load Task 11 similarity graph
G = nx.read_gml('./task11_similarity_graph.gml')
print(f"Loaded graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Debug: Check edge weight distribution
weights = [G[u][v]['weight'] for u, v in G.edges()]
print(f"Edge weight stats: min={min(weights):.4f}, max={max(weights):.4f}, mean={np.mean(weights):.4f}, std={np.std(weights):.4f}")

# Optionally load Task 11 communities and Task 2 DataFrame
comm_df = None
df = None
# if Path('./task11_communities.csv').exists():
comm_df = pd.read_csv('./task11_communities.csv')
print(f"Loaded communities: {len(comm_df)} restaurants")
# if Path('./task2_df.parquet').exists():
df = pd.read_parquet('./task2_df.parquet')
print(f"Loaded DataFrame: {len(df)} rows")

# Step 1: Disparity filtering
def disparity_filter(G, alpha=0.4):
    """Apply disparity filtering to extract the backbone."""
    G_backbone = G.copy()
    edges_to_remove = []
    
    for node in G.nodes():
        # Get degree and edge weights
        degree = G.degree(node)
        if degree <= 1:
            continue
        # Sum of weights for the node
        strength = sum(G[node][neighbor]['weight'] for neighbor in G.neighbors(node))
        # Compute normalized weights
        for neighbor in G.neighbors(node):
            weight = G[node][neighbor]['weight']
            p_ij = weight / strength
            # Disparity filter: keep edges where weight is significant
            # Null model: integral from p_ij to 1 of (1-x)^(k-1) < alpha
            if degree > 1:
                integral = (1 - p_ij) ** (degree - 1)
                if integral > alpha:
                    edges_to_remove.append((node, neighbor))
    
    print(f"Edges before filtering: {G.number_of_edges()}")
    print(f"Edges to remove: {len(edges_to_remove)}")
    G_backbone.remove_edges_from(edges_to_remove)
    # Remove isolated nodes
    isolated = list(nx.isolates(G_backbone))
    G_backbone.remove_nodes_from(isolated)
    
    return G_backbone

alpha = 0.4
G_backbone = disparity_filter(G, alpha)
print(f"Backbone graph: {G_backbone.number_of_nodes()} nodes, {G_backbone.number_of_edges()} edges")

# Check for empty backbone
if G_backbone.number_of_nodes() == 0:
    print("Warning: Backbone is empty. Try increasing alpha or checking input graph weights.")

# Step 2: Analyze the backbone
# Compute metrics
num_nodes = G_backbone.number_of_nodes()
num_edges = G_backbone.number_of_edges()
avg_degree = sum(dict(G_backbone.degree()).values()) / num_nodes if num_nodes > 0 else 0
num_components = nx.number_connected_components(G_backbone) if num_nodes > 0 else 0

# Degree distribution
degrees = [degree for _, degree in G_backbone.degree()]
avg_degree_dist = np.mean(degrees) if degrees else 0
std_degree_dist = np.std(degrees) if degrees else 0

# Save analysis
analysis_data = {
    'metric': ['num_nodes', 'num_edges', 'avg_degree', 'num_components', 'avg_degree_dist', 'std_degree_dist'],
    'value': [num_nodes, num_edges, avg_degree, num_components, avg_degree_dist, std_degree_dist]
}
analysis_df = pd.DataFrame(analysis_data)
print("\nBackbone Analysis:")
print(analysis_df)

# Step 3: Visualize the backbone
def visualize_backbone(graph, comm_df, df, output_file, sample_size=300):
    """Visualize a sampled backbone subgraph with PyVis."""
    if graph.number_of_nodes() == 0:
        print("Cannot visualize: Backbone graph is empty.")
        return
    
    # Sample nodes
    sample_nodes = list(graph.nodes())[:min(sample_size, graph.number_of_nodes())]
    G_sample = graph.subgraph(sample_nodes)
    print(f"Visualization subgraph: {G_sample.number_of_nodes()} nodes, {G_sample.number_of_edges()} edges")
    
    net = Network(notebook=True, height="600px", width="100%", directed=False, cdn_resources='in_line')
    
    # Color by community if available, else by degree
    if comm_df is not None:
        partition = dict(zip(comm_df['restaurant_name'], comm_df['community']))
        unique_comms = set(partition.values())
        n_comms = len(unique_comms)
        color_map = plt.colormaps['viridis']
        comm_to_color = {comm: mcolors.rgb2hex(color_map(i / n_comms)) for i, comm in enumerate(unique_comms)}
    else:
        degrees = dict(graph.degree())
        max_degree = max(degrees.values()) if degrees else 1
        color_map = plt.colormaps['viridis']
        comm_to_color = {deg: mcolors.rgb2hex(color_map(deg / max_degree)) for deg in set(degrees.values())}
    
    # Node size by degree
    degrees = dict(graph.degree())
    max_degree = max(degrees.values()) if degrees else 1
    min_size, max_size = 10, 50
    
    # Node attributes
    for node in G_sample.nodes():
        degree = degrees.get(node, 0)
        size = min_size + (max_size - min_size) * (degree / max_degree) if max_degree > 0 else 20
        if comm_df is not None and node in partition:
            color = comm_to_color.get(partition[node], '#888888')
            title = f"Restaurant: {node}\nCommunity: {partition[node]}\nDegree: {degree}"
        else:
            color = comm_to_color.get(degree, '#888888')
            title = f"Restaurant: {node}\nDegree: {degree}"
        net.add_node(
            node,
            label=node,
            color=color,
            size=size,
            title=title
        )
    
    # Edge attributes
    for edge in G_sample.edges(data=True):
        weight = edge[2].get('weight', 1)
        net.add_edge(
            edge[0],
            edge[1],
            value=weight * 2,
            title=f"Similarity: {weight:.2f}",
            color='#888888'
        )
    
    net.show_buttons(filter_=['physics'])
    net.show(output_file)
    print(f"Visualization saved to {output_file}")

visualize_backbone(G_backbone, comm_df, df, "./task12_backbone_network.html")

# Step 4: Save outputs
nx.write_gml(G_backbone, './task12_backbone_graph.gml')
analysis_df.to_csv('./task12_backbone_analysis.csv', index=False)
print("Saved backbone graph to ./task12_backbone_graph.gml")
print("Saved analysis to ./task12_backbone_analysis.csv")


[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Loaded graph: 4846 nodes, 1228947 edges
Edge weight stats: min=0.3000, max=1.0000, mean=0.5121, std=0.1799
Loaded communities: 4846 restaurants
Loaded DataFrame: 5000 rows
Edges before filtering: 1228947
Edges to remove: 1147449
Backbone graph: 4175 nodes, 528199 edges

Backbone Analysis:
            metric          value
0        num_nodes    4175.000000
1        num_edges  528199.000000
2       avg_degree     253.029461
3   num_components       2.000000
4  avg_degree_dist     253.029461
5  std_degree_dist     178.524334
Visualization subgraph: 300 nodes, 2546 edges
./task12_backbone_network.html
Visualization saved to ./task12_backbone_network.html
Saved backbone graph to ./task12_backbone_graph.gml
Saved analysis to ./task12_backbone_analysis.csv


In [1]:
from IPython.display import IFrame
def showMyHtmlGraph(path):
    display(IFrame(path, width="100%", height="600px"))
showMyHtmlGraph("./task12_backbone_network.html")

# task 13
## Recommendation via Network Proximity 
- For a given restaurant, suggest similar ones based on shortest paths, common neighbors, or graph 
embeddings. 
- Evaluate how well these recommendations align with shared features or cuisines. 


In [2]:
# Install required package
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 222.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 53.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 32.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.0
    Uninstalling scipy-1.14.0:
      Successfully uninstalled scipy-1.14.0

[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from node2vec import Node2Vec
# Load graph (prefer backbone, fallback to similarity graph)
graph_file = './task12_backbone_graph.gml'
# if Path(graph_file).exists():
G = nx.read_gml(graph_file)
print(f"Loaded backbone graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
# else:
#     graph_file = './task11_similarity_graph.gml'
#     G = nx.read_gml(graph_file)
#     print(f"Loaded similarity graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Check if graph is empty
if G.number_of_nodes() == 0:
    print("Error: Graph is empty. Please ensure Task 12 produced a valid graph.")
    exit()

# Load Task 2 DataFrame for features
df = None
# if Path('./task2_df.parquet').exists():
df = pd.read_parquet('./task2_df.parquet')
print(f"Loaded DataFrame: {len(df)} rows")
# else:
#     print("Error: task2_df.parquet not found. Required for evaluation.")
#     exit()

# Load Task 11 communities (optional)
comm_df = None
# if Path('./task11_communities.csv').exists():
comm_df = pd.read_csv('./task11_communities.csv')
print(f"Loaded communities: {len(comm_df)} restaurants")

# Step 1: Generate Node2Vec embeddings
def generate_embeddings(G, dimensions=64, walk_length=30, num_walks=200, workers=4):
    """Generate Node2Vec embeddings for the graph."""
    node2vec = Node2Vec(
        G,
        dimensions=dimensions,
        walk_length=walk_length,
        num_walks=num_walks,
        workers=workers,
        quiet=True
    )
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    embeddings = {node: model.wv[node] for node in G.nodes()}
    return embeddings

print("Generating Node2Vec embeddings...")
embeddings = generate_embeddings(G)

# Step 2: Recommendation functions
def shortest_path_recommendations(G, source, top_k=5):
    """Recommend restaurants based on shortest path lengths."""
    try:
        lengths = nx.single_source_dijkstra_path_length(G, source, weight='weight')
        # Filter restaurants within distance 2 (avoid self and disconnected)
        candidates = {n: 1.0 / (length + 1e-6) for n, length in lengths.items() if n != source and length <= 2}
        # Sort by score (inverse distance)
        sorted_candidates = sorted(candidates.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return [(n, score) for n, score in sorted_candidates]
    except nx.NetworkXNoPath:
        return []

def common_neighbors_recommendations(G, source, top_k=5):
    """Recommend restaurants based on common neighbors (Jaccard similarity)."""
    candidates = []
    for node in G.nodes():
        if node == source:
            continue
        # Get neighbors
        source_neighbors = set(G.neighbors(source))
        node_neighbors = set(G.neighbors(node))
        # Compute Jaccard similarity
        intersection = len(source_neighbors & node_neighbors)
        union = len(source_neighbors | node_neighbors)
        jaccard = intersection / union if union > 0 else 0
        if jaccard > 0:
            candidates.append((node, jaccard))
    # Sort by Jaccard similarity
    sorted_candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:top_k]
    return sorted_candidates

def embedding_recommendations(embeddings, source, nodes, top_k=5):
    """Recommend restaurants based on embedding cosine similarity."""
    if source not in embeddings:
        return []
    source_embedding = embeddings[source].reshape(1, -1)
    candidate_embeddings = np.array([embeddings[node] for node in nodes if node != source])
    candidate_nodes = [node for node in nodes if node != source]
    if not candidate_nodes:
        return []
    similarities = cosine_similarity(source_embedding, candidate_embeddings)[0]
    # Sort by similarity
    sorted_indices = np.argsort(similarities)[::-1][:top_k]
    return [(candidate_nodes[i], similarities[i]) for i in sorted_indices]

def hybrid_recommendations(G, embeddings, source, top_k=5, weights={'sp': 0.3, 'cn': 0.3, 'emb': 0.4}):
    """Combine recommendations from all methods."""
    # Get recommendations from each method
    sp_recs = shortest_path_recommendations(G, source, top_k=top_k)
    cn_recs = common_neighbors_recommendations(G, source, top_k=top_k)
    emb_recs = embedding_recommendations(embeddings, source, G.nodes(), top_k=top_k)
    
    # Normalize scores
    def normalize_scores(recs):
        if not recs:
            return recs
        max_score = max(score for _, score in recs) if recs else 1
        return [(n, score / max_score) for n, score in recs]
    
    sp_recs = normalize_scores(sp_recs)
    cn_recs = normalize_scores(cn_recs)
    emb_recs = normalize_scores(emb_recs)
    
    # Combine scores
    score_dict = {}
    for node, score in sp_recs:
        score_dict[node] = score_dict.get(node, 0) + weights['sp'] * score
    for node, score in cn_recs:
        score_dict[node] = score_dict.get(node, 0) + weights['cn'] * score
    for node, score in emb_recs:
        score_dict[node] = score_dict.get(node, 0) + weights['emb'] * score
    
    # Sort by combined score
    sorted_recs = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return [(node, score, 'hybrid') for node, score in sorted_recs]

# Step 3: Generate recommendations for a sample of restaurants
sample_restaurants = list(G.nodes())[:10]  # Test with 10 restaurants
recommendations = []

print("Generating recommendations...")
for restaurant in sample_restaurants:
    # Get recommendations from each method
    sp_recs = [(r, s, 'shortest_path') for r, s in shortest_path_recommendations(G, restaurant)]
    cn_recs = [(r, s, 'common_neighbors') for r, s in common_neighbors_recommendations(G, restaurant)]
    emb_recs = [(r, s, 'embeddings') for r, s in embedding_recommendations(embeddings, restaurant, G.nodes())]
    hybrid_recs = hybrid_recommendations(G, embeddings, restaurant)
    
    # Combine all recommendations
    recommendations.extend([(restaurant, r, s, m) for r, s, m in sp_recs + cn_recs + emb_recs + hybrid_recs])

# Save recommendations
rec_df = pd.DataFrame(recommendations, columns=['restaurant', 'recommended', 'score', 'method'])
rec_df.to_csv('./task13_recommendations.csv', index=False)
print("Saved recommendations to ./task13_recommendations.csv")

# Step 4: Evaluate recommendations
def evaluate_recommendations(rec_df, feature_df):
    """Evaluate recommendations based on cuisine overlap and feature similarity."""
    eval_results = []
    
    # Ensure feature_df has required columns
    if 'cuisine' not in feature_df.columns or 'average_rating' not in feature_df.columns:
        print("Warning: feature_df missing 'cuisine' or 'average_rating'. Skipping evaluation.")
        return pd.DataFrame()
    
    # Group by restaurant and method
    grouped = rec_df.groupby(['restaurant', 'method'])
    
    for (restaurant, method), group in grouped:
        recommended = group['recommended'].tolist()
        if not recommended:
            continue
        
        # Get cuisine of source restaurant
        source_row = feature_df[feature_df['restaurant_name'] == restaurant]
        if source_row.empty:
            continue
        source_cuisine = source_row['cuisine'].iloc[0]
        
        # Cuisine overlap
        rec_rows = feature_df[feature_df['restaurant_name'].isin(recommended)]
        same_cuisine = rec_rows[rec_rows['cuisine'] == source_cuisine]
        cuisine_overlap = len(same_cuisine) / len(rec_rows) if len(rec_rows) > 0 else 0
        
        # Feature similarity
        numerical_features = ['average_rating', 'total_reviews_count']
        if 'total_reviews_count' not in feature_df.columns:
            numerical_features = ['average_rating']
        
        source_features = source_row[numerical_features].values
        rec_features = rec_rows[numerical_features].values
        if len(rec_features) > 0 and source_features.shape[0] > 0:
            similarities = cosine_similarity(source_features, rec_features)[0]
            avg_similarity = np.mean(similarities)
        else:
            avg_similarity = 0
        
        eval_results.append({
            'restaurant': restaurant,
            'method': method,
            'cuisine_overlap': cuisine_overlap,
            'feature_similarity': avg_similarity,
            'num_recommendations': len(recommended)
        })
    
    return pd.DataFrame(eval_results)

# Evaluate and save
eval_df = evaluate_recommendations(rec_df, df)
eval_df.to_csv('./task13_evaluation.csv', index=False)
print("Saved evaluation to ./task13_evaluation.csv")

# Print evaluation summary
if not eval_df.empty:
    print("\nEvaluation Summary:")
    print(eval_df.groupby('method')[['cuisine_overlap', 'feature_similarity']].mean())
else:
    print("No evaluation results generated.")

Loaded backbone graph: 4175 nodes, 528199 edges
Loaded DataFrame: 5000 rows
Loaded communities: 4846 restaurants
Generating Node2Vec embeddings...
